<a href="https://colab.research.google.com/github/nickprock/corso_data_science/blob/devs/machine_learning_pills/01_supervised/02_classificare_lo_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classificare lo spam

<br>

![spam](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fstorage.googleapis.com%2Fcdn-leanplum-images%2F1%2F2019%2F01%2FEmail-marketing-spam-feature-min.jpg&f=1&nofb=1)

<br>

[Image Credits](https://www.leanplum.com/blog/email-marketing-campaign-spam/)

<br>

La classificazione delle e-mail di spam è stato uno dei primi problemi "di massa" su cui si è cimentato il machine learning.

Per individuare i messaggi di spam vengono utilizzati i classificatori binari, una tipo di modelli che discriminano tra due classi, nel nostro caso:
* SPAM
* NON SPAM

Questi fanno parte dei modelli **supervisionati** avendo bisogno di esempi etichettati per addestrare i modelli ed è un problema di **classificazione** visto che la variabile target è binaria.

Questo notebook è derivato da un notebook di [Valentina Porcu](https://github.com/valentinap), tra i link utili vi lascerò il suo corso di Data Science in Python su Udemy.

### NLTK
[Natural Language ToolKit](https://www.nltk.org) è una libreria vastissima per il NLP, la useremo in fase di pulizia del testo perchè contiene molti metodi lavorare sui testi ed è facile da installare e utilizzare e supporta moltissime lingue avendo oltre 50 lessici.

In [0]:
import pandas as pd
import nltk

### Dataset

Il dataset utilizzato è [SMS SPAM Collection](http://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) disponibile dal repositori dell'[UCI Machine Learning](https://archive.ics.uci.edu/ml/index.php).

Ha solo due colonne:
* **label**: {spam, ham}
* **text**: il testo del messaggio

### Caricamento del dataset

***Se stai usando il notebook su Colab esegui le prossime due celle, altrimenenti vai direttamente al caricamento con *read_csv* inserendo il path del tuo file *SMSSpamCollection.csv***

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/YOUR_CODE'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('SMSSpamCollection')

In [0]:
df = pd.read_csv('SMSSpamCollection', sep = "\t", names=["label", "text"])

In [0]:
print(df.head())

Utilizzando la funzione describe per i gruppi possiamo vedere che:
* ci sono 5572 messaggi, di cui 5169 unici
* lo spam è il 13.4% del dataset
* il messaggio di spam più comune invita a contattare il servizio clienti.

In [0]:
print(df.groupby('label').describe())

### Preprocessing

<br>

![nlp_prep](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.kdnuggets.com%2Fwp-content%2Fuploads%2Ftext-preprocessing-framework-2.png&f=1&nofb=1)

<br>

[Image Credits](https://www.kdnuggets.com/2017/12/general-approach-preprocessing-text-data.html)

<br>

Il preprocessing dei testi ha delle sue particolari sequenze di operazioni per renderlo "appetibile" agli algoritmi. Per eseguirle useremo nltk.

La pulizia si basa su:
* sostituire le maiuscole con le minuscole così da uniformare i termini
* eliminare numeri, punteggiatura, caratteri speciali
* eliminare le stopwords, ovvero quelle parole che appaiono molto spesso nel testo ma non danno particolari informazioni (es. articoli, congiunzioni, ...)
* applicare lo stemming, ovvero ridurre le parole alla loro radice così da no avere termini differenti per singolare/plurale, maschile/femminile, coniugazione di verbi.

Prima di condurre qualsiasi operazione sui testi però trasformiamo la label da stringa a valore numerico:
* ham: 0
* spam: 1

In [0]:
cl = {'ham': 0, 'spam': 1}
df['label'] = df['label'].map(cl)

In [0]:
print(df.head())

In [0]:
nltk.download('stopwords')
sw = set(nltk.corpus.stopwords.words('english'))

In [0]:
print(sw)

In [0]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, df.shape[0]):
    text = re.sub('[^a-zA-Z]', ' ', df['text'][i]) # mantieni solo il testo
    text = text.lower() # tutto minuscolo
    text = text.split() # ogni frase un vettore di parole
    ps = PorterStemmer() # stemmer
    text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))] # elimina le stopword e applica lo stemming
    text = ' '.join(text) # ricostruisce la frase
    corpus.append(text) # ricostruisce il dataset

In [0]:
corpus[:5]

### Bag of Words

Per passare il testo agli algoritmi dobbiamo trasformarlo in una matrice, in questo caso utilizziamo la tacnica del **bag-of-words**.
Viene creato un dizionario di termini, pero gni frase si annota la presenza/assenza del termine e la frequenza, ogni frase sarà la riga di una *Document - Term Matrix*.

<br>

![bow](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fcdn-images-1.medium.com%2Fmax%2F1600%2F0*jdeMyO7-xpbvvi4b.jpg&f=1&nofb=1)

<br>

[Image Credits](https://medium.com/greyatom/an-introduction-to-bag-of-words-in-nlp-ac967d43b428)

<br>

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
cv = CountVectorizer(max_features = 2000) # per comodità il nostro vocabolario conterrà solo i 2000 termini più frequenti
x = cv.fit_transform(corpus).toarray()
cl = df['label'].values

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, cl, test_size = 0.3, random_state = 42)

### Regressione Logistica

La regressione logistica, è un modello di regressione non lineare utilizzato quando la variabile dipendente è di tipo dicotomico.
A dispetto del nome è un modello di classificazione binaria infatti il suo output è una probabilità di appartenenza/non appartenenza ad una categoria.

Viene considerato come l'entry level del Machine Learning, ed è utilizzato soprattutto per:
* fraud detection
* predictive maintenance
* classificazione di documenti


<br>

![penguin](https://miro.medium.com/max/800/1*UgYbimgPXf6XXxMy2yqRLw.png)

<br>

[Image Credits](http://dataaspirant.com/2017/03/02/how-logistic-regression-model-works/)

<br>

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
lr = LogisticRegression()

In [0]:
lr.fit(x_train, y_train)

In [0]:
lr_pred = lr.predict(x_test)

### Valutazione del modello

Per la valutazione del modello di classificazione abbiamo diversi strumenti e diverse misure.

Il primo strumento è la **confusion matrix**.

<br>

![cm](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.stack.imgur.com%2FysM0Z.png&f=1&nofb=1)

<br>

[Image Credits](https://tex.stackexchange.com/questions/20267/how-to-construct-a-confusion-matrix-in-latex)

<br>

La confusion matrix mette in relazione i valori osservati con quelli stimati dal modello ed è lo strumento più immediato di valutazione in quanto ci permette di calcolare le seguenti metrriche:
* accuratezza (accuracy): (TP + TN)/TotalCase ovvero i casi in cui il modello ha predetto bene, fratto i casi totali. Questa è la misura più utilizzata ma attenzione perchè per dataset sbilanciati, anche più del nostro, pur non facendo stime corrette si può arrivare ad una accuratezza molto alta.
* precisione (precision): TP/(TP + FP) ovvero quanto è preciso accurato il nostro modello quando stima una classe, quanti elementi tra gli stimati ne fanno effettivamente parte? Ad esempio se etichetta come spam un'email non spam, l'utente potrebbe perdere informazioni.
* richiamo (recall): TP/(TP + FN), misura il *costo del modello*, ad esempio misura quante attività fraudolente vengono classificate come non-fraudolente.
* F1: 2 x [(Precision x Recall)/(Precision + recall)], è una misura che valuta l'equilibrio tra le precedenti, è usata spesso per valutare modelli diversi.

In [0]:
from sklearn.metrics import confusion_matrix, classification_report

In [0]:
print(confusion_matrix(y_test, lr_pred))

In [0]:
print(classification_report(y_test, lr_pred))

In questo caso l'accuratezza è buona, 98% perchè solo 29 casi su 1672 vengono classificati male.

La precisione non è male, abbiamo solo 3 e-mail non spam che vengono etichettate come spam.

Cosa diversa per la recall, in questo caso il filtro anti-spam, non funziona bene, ci sono sfuggite 26 e-mail!

La F1-score totale del modello è 96% ma risente dello sbilanciamento del dataset, quindi valutando solo quella sull'etichetta spam abbiamo il 93%.

### Support Vector Machines

Le Support Vector Machine sono un modello di classificazione/regressione che basano il loro funzionamento su delle funzioni *kernel* che suddiviono lo spazio in cui sono presenti le osservazioni. Funzionano molto bene per classificazioni binarie, sono state utilizzate anche per altri tipi di classificazioni e per la regressione ma non con gli stessi risultati.

Nel nostro caso useremo una funzione kernel lineare.

<br>

![svm](https://upload.wikimedia.org/wikipedia/commons/thumb/2/2a/Svm_max_sep_hyperplane_with_margin.png/360px-Svm_max_sep_hyperplane_with_margin.png)

<br>

[Image Credits](https://en.wikipedia.org/wiki/Support-vector_machine)

<br>



In [0]:
from sklearn.linear_model import SGDClassifier

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
clf = SGDClassifier()

In [0]:
clf.fit(x_train, y_train)

In [0]:
clf_pred = clf.predict(x_test)

### Valutazione del modello

In [0]:
print(confusion_matrix(y_test, clf_pred))

In [0]:
print(classification_report(y_test, clf_pred))

Lascio a voi i commenti

### Naive Bayes

<br>

![Bayes](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fthatware.co%2Fwp-content%2Fuploads%2F2019%2F03%2Fnaive-bayes.png&f=1&nofb=1)

<br>

[Image Credits](https://thatware.co/naive-bayes/)

<br>

Il naive bayes è un classificatore molto semplice che si basa sull'omonimo teorema.

Per le ipotesi abbastanza semplici alla sua base è molto utile per piccoli set di dati e viene usato spesso nella classificazione dei documenti.

Con gli iperparamentri opportunamente regolati può competere con modelli più complessi come le SVM.

In scikit-learn sono presenti diversi classificatori bayesiani a seconda della distribuzione del dato, qui ne testeremo 2, vi anticipo già che il *MultinomialNB* è quello più utilizzato per la classificazione dei documenti.


##### GaussianNB

In [0]:
from sklearn.naive_bayes import GaussianNB

In [0]:
nb = GaussianNB()

In [0]:
nb.fit(x_train, y_train)

In [0]:
nb_pred = nb.predict(x_test)

##### MultinomialNB

In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
nb2 = MultinomialNB()

In [0]:
nb2.fit(x_train, y_train)

In [0]:
nb2_pred = nb2.predict(x_test)

#### Valutazione dei modelli

In [0]:
print(confusion_matrix(y_test, nb_pred))

In [0]:
print(classification_report(y_test, nb_pred))

In [0]:
print(confusion_matrix(y_test, nb2_pred))

In [0]:
print(classification_report(y_test, nb2_pred))

Lascio a voi la valutazione del modello

### Conclusioni

Ora è il momento di scegliere il classificatore migliore per questo problema, qual è?

Come potrebbe essere migliorata la prestazione?

Come potremmo valutare se siamo in una situazione di overfitting?

## Link Utili

[A general approach to preprocessing text data](https://www.kdnuggets.com/2017/12/general-approach-preprocessing-text-data.html)

[Corso completo di data science con Python](https://www.udemy.com/course/data-science-con-python/)

[Introduction to bag of words](https://medium.com/greyatom/an-introduction-to-bag-of-words-in-nlp-ac967d43b428)

[Gentle introduction to bag of words](https://machinelearningmastery.com/gentle-introduction-bag-words-model/)

[CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

[Accuracy, Precision, Recall, F1](https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9)

[Naive Bayes](https://machinelearningmastery.com/naive-bayes-for-machine-learning/)